## Convolution 

1. https://machinelearningmastery.com/convolutional-layers-for-deep-learning-neural-networks/
2. https://towardsdatascience.com/simple-introduction-to-convolutional-neural-networks-cdf8d3077bac
3. https://youtu.be/HGwBXDKFk9I

## Import packages

In [2]:
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedKFold

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
import torchvision.transforms as transforms

from fastprogress import progress_bar, master_bar
import mlcrate.time as mlctime


from sklearn.metrics import f1_score


timer = mlctime.Timer()

In [3]:
### Dataset downloaded from - https://www.kaggle.com/datasets/zalando-research/fashionmnist

In [4]:
train_df = pd.read_csv("../../input/fashionmnist/fashion-mnist_train.csv")
test_df = pd.read_csv("../../input/fashionmnist/fashion-mnist_test.csv")
train_df.shape, test_df.shape

((60000, 785), (10000, 785))

In [5]:
train_df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## CV Split

In [6]:
n_folds = 5 # number of folds
seed = 42 # seed to control randomness

nFolds = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=seed)
for n, (train_index, val_index) in enumerate(nFolds.split(X=train_df, y=train_df['label'])):
    train_df.loc[val_index, f'fold_{n_folds}_seed_{seed}'] = int(n)

train_df[f'fold_{n_folds}_seed_{seed}'] = train_df[f'fold_{n_folds}_seed_{seed}'].astype(int)

print(train_df.groupby(f'fold_{n_folds}_seed_{seed}').size())

fold_5_seed_42
0    12000
1    12000
2    12000
3    12000
4    12000
dtype: int64


## Pytorch Dataset

In [7]:
class FashionDataset(Dataset):
    """User defined class to build a datset using Pytorch class Dataset."""
    
    def __init__(self, data, transform = None):
        """Method to initilaize variables.""" 
        self.fashion_MNIST = list(data.values)
        self.transform = transform
        
        label = []
        image = []
        
        for i in self.fashion_MNIST:
             # first column is of labels.
            label.append(i[0])
            image.append(i[2:])
        self.labels = np.asarray(label)
        # Dimension of Images = 28 * 28 * 1. where height = width = 28 and color_channels = 1.
        self.images = np.asarray(image).reshape(-1, 28, 28, 1).astype('float32')

    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        label = self.labels[index]
        image = self.images[index]

        if self.transform is not None:
            image = self.transform(image)
        
        image = torch.cat((image, image, image), dim=0)

        return image, label


## DEVICE

In [8]:
device = torch.device("cuda")

## MODEL 

### Tutorial on how to finetune torchvision pretrained models
1. https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html

In [9]:
dir(models)

['AlexNet',
 'DenseNet',
 'EfficientNet',
 'GoogLeNet',
 'GoogLeNetOutputs',
 'Inception3',
 'InceptionOutputs',
 'MNASNet',
 'MobileNetV2',
 'MobileNetV3',
 'RegNet',
 'ResNet',
 'ShuffleNetV2',
 'SqueezeNet',
 'VGG',
 '_GoogLeNetOutputs',
 '_InceptionOutputs',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_utils',
 'alexnet',
 'densenet',
 'densenet121',
 'densenet161',
 'densenet169',
 'densenet201',
 'detection',
 'efficientnet',
 'efficientnet_b0',
 'efficientnet_b1',
 'efficientnet_b2',
 'efficientnet_b3',
 'efficientnet_b4',
 'efficientnet_b5',
 'efficientnet_b6',
 'efficientnet_b7',
 'feature_extraction',
 'googlenet',
 'inception',
 'inception_v3',
 'mnasnet',
 'mnasnet0_5',
 'mnasnet0_75',
 'mnasnet1_0',
 'mnasnet1_3',
 'mobilenet',
 'mobilenet_v2',
 'mobilenet_v3_large',
 'mobilenet_v3_small',
 'mobilenetv2',
 'mobilenetv3',
 'quantization',
 'regnet',
 'regnet_x_16gf',
 'regnet_x_1_6gf',
 're

In [10]:
model = models.resnet18(pretrained=True)
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

## Modify the linear layer of the model based on the customized use case

In [11]:
num_labels = train_df['label'].nunique()
num_labels

10

In [12]:
model.fc = nn.Linear(512, num_labels)
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [13]:
model.to(device);

## METRICS

In [14]:
## Calculate metrics
def get_metrics(y_true, y_pred):
    f1 = f1_score(y_true, y_pred, average="weighted")
    return f1

In [ ]:
for fold in range(n_folds):
    print(f"------------- Fold {fold+1} -------------")

    # TRAIN & VALID DATA
    train_set = train_df[train_df[f'fold_{n_folds}_seed_{seed}'] != fold]
    valid_set = train_df[train_df[f'fold_{n_folds}_seed_{seed}'] == fold]
    
    train_set.drop(f'fold_{n_folds}_seed_{seed}', axis=1, inplace=True)
    valid_set.drop(f'fold_{n_folds}_seed_{seed}', axis=1, inplace=True)

    print(f"No.of samples in train data: {train_set.shape[0]}")
    print(f"No.of samples in valid data: {valid_set.shape[0]}")    

    # DATASET
    train_dataset = FashionDataset(train_df, transform=transforms.Compose([transforms.ToTensor()]))
    valid_dataset = FashionDataset(train_df, transform=transforms.Compose([transforms.ToTensor()]))
    print(f"TRAIN Dataset: {train_dataset.__len__()}")
    print(f"VALID Dataset: {valid_dataset.__len__()}")
    
    # DATALOADER
    train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    valid_dataloader = DataLoader(valid_dataset, batch_size=64, shuffle=False)
    
    # optimizer
    optimizer = optim.SGD(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()
    
    mb = master_bar(range(5))
    for epoch in mb:  
        # ------------------------------------ Training loop ----------------------------------------
        timer.add('train')
        tr_loss, tr_f1_score, tr_steps = 0, 0, 0
        for bi, d in enumerate(progress_bar(train_dataloader, parent=mb)):
            model.train()
            images = d[0]
            labels = d[1]

            # load input data to device
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)

            preds = outputs
            
            loss = criterion(preds, labels)  
            
            logits = torch.argmax(torch.softmax(preds, dim=-1), dim=-1)
            
            labels = labels.to('cpu')
            logits = logits.detach().to('cpu')

            f1 = get_metrics(labels, logits)

            tr_loss += loss.item()
            tr_f1_score += f1
            
            tr_steps += 1  

            avg_tr_loss = tr_loss/tr_steps
            avg_tr_f1_score = tr_f1_score/tr_steps
            
            mb.child.comment = 'tr_loss: {:.4f}; tr_f1_score: {:.4f}; avg_tr_loss: {:.4f}; avg_tr_f1_score: {:.4f}'.format(loss.item(), f1, avg_tr_loss, avg_tr_f1_score) 
        
        tr_time = timer.fsince('train')
        
        # ------------------------------------------ Validation loop ---------------------------------------
        timer.add('val')
        vl_loss, vl_f1_score, vl_steps = 0, 0, 0
        for bi, d in enumerate(progress_bar(valid_dataloader, parent=mb)):
            model.eval()
            images = d[0]
            labels = d[1]

            # load input data to device
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)

            preds = outputs
            
            loss = criterion(preds, labels)  
            
            logits = torch.argmax(torch.softmax(preds, dim=-1), dim=-1)
            
            labels = labels.to('cpu')
            logits = logits.detach().to('cpu')

            f1 = get_metrics(labels, logits)

            vl_loss += loss.item()
            vl_f1_score += f1
            
            vl_steps += 1  

            avg_vl_loss = vl_loss/vl_steps
            avg_vl_f1_score = vl_f1_score/vl_steps
            
            mb.child.comment = 'vl_loss: {:.4f}; vl_f1_score: {:.4f}; avg_vl_loss: {:.4f}; avg_vl_f1_score: {:.4f}'.format(loss.item(), f1, avg_vl_loss, avg_vl_f1_score) 
        
        vl_time = timer.fsince('val')
        
        output = f"Train_time : {tr_time} - Val_time : {vl_time} - Ep : {epoch} - Bi : {bi} - Loss : {avg_tr_loss:.4f}; {avg_vl_loss:.4f} - f1 : {avg_tr_f1_score:.4f}; {avg_vl_f1_score:.4f}"
        print(f"{output}")


------------- Fold 1 -------------
No.of samples in train data: 48000
No.of samples in valid data: 12000


/home/sthueva/anaconda3/envs/kaggle/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


TRAIN Dataset: 60000
VALID Dataset: 60000


Train_time : 3s - Val_time : 3s - Ep : 0 - Bi : 937 - Loss : 2.6413; 2.6824 - f1 : 0.0800; 0.0764
Train_time : 3s - Val_time : 3s - Ep : 1 - Bi : 937 - Loss : 2.6399; 2.6593 - f1 : 0.0810; 0.0797
Train_time : 3s - Val_time : 3s - Ep : 2 - Bi : 937 - Loss : 2.6400; 2.6619 - f1 : 0.0817; 0.0775
Train_time : 3s - Val_time : 3s - Ep : 3 - Bi : 937 - Loss : 2.6389; 2.6891 - f1 : 0.0799; 0.0792
Train_time : 3s - Val_time : 3s - Ep : 4 - Bi : 937 - Loss : 2.6404; 2.6712 - f1 : 0.0810; 0.0791
------------- Fold 2 -------------
No.of samples in train data: 48000
No.of samples in valid data: 12000


/home/sthueva/anaconda3/envs/kaggle/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


...

TRAIN Dataset: 60000
VALID Dataset: 60000


Train_time : 3s - Val_time : 3s - Ep : 0 - Bi : 937 - Loss : 2.6376; 2.6752 - f1 : 0.0798; 0.0797
Train_time : 3s - Val_time : 3s - Ep : 1 - Bi : 937 - Loss : 2.6424; 2.6549 - f1 : 0.0816; 0.0803
Train_time : 3s - Val_time : 3s - Ep : 2 - Bi : 937 - Loss : 2.6403; 2.6467 - f1 : 0.0791; 0.0807
Train_time : 3s - Val_time : 3s - Ep : 3 - Bi : 937 - Loss : 2.6420; 2.6607 - f1 : 0.0795; 0.0779
Train_time : 3s - Val_time : 3s - Ep : 4 - Bi : 937 - Loss : 2.6385; 2.6734 - f1 : 0.0813; 0.0786
------------- Fold 5 -------------
No.of samples in train data: 48000
No.of samples in valid data: 12000


/home/sthueva/anaconda3/envs/kaggle/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


TRAIN Dataset: 60000
VALID Dataset: 60000


Train_time : 3s - Val_time : 3s - Ep : 0 - Bi : 937 - Loss : 2.6395; 2.6829 - f1 : 0.0809; 0.0768
Train_time : 3s - Val_time : 3s - Ep : 1 - Bi : 937 - Loss : 2.6391; 2.6542 - f1 : 0.0799; 0.0822
Train_time : 3s - Val_time : 3s - Ep : 2 - Bi : 937 - Loss : 2.6404; 2.6470 - f1 : 0.0809; 0.0792
Train_time : 3s - Val_time : 3s - Ep : 3 - Bi : 937 - Loss : 2.6411; 2.6609 - f1 : 0.0796; 0.0791
